## Class decorators

In [1]:
def savings(cls):
    cls.account_type = 'savings'
    return cls

def checking(cls):
    cls.account_type = 'checking'
    
    
class Account:
    pass


@savings
class Bank1Savings(Account):
    pass

@savings
class Bank2Savings(Account):
    pass

@checking
class Bank1Checking(Account):
    pass

@checking
class Bank2Checking(Account):
    pass

### The above works but is a bit repetative. We can create a parameterized decorator instead

In [ ]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

@account_type('Savings')
class Bank1Savings(Account):
    pass

@account_type('Checking')
class Bank1Checking(Account):
    pass

### We are not restricted to just adding data atttributes. We can also use our decorator to add functions

In [3]:
def hello(cls):
    cls.hello = lambda self: f'{self} says hello'
    return cls

@hello
class Person:
    
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name
    

p = Person('John')
p.hello()
    

'John says hello'

In [6]:
from functools import wraps

def func_logger(fn):
    
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

class Person:
    
    @func_logger
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    @func_logger
    def greet(self):
        return f'Hello, my name is {self.name}, and I am {self.age} years old'
    
p = Person('John', 48)
p.greet()

Log: Person.__init__((<__main__.Person object at 0x000001D0830CD1F0>, 'John', 48), {}) = None
Log: Person.greet((<__main__.Person object at 0x000001D0830CD1F0>,), {}) = Hello, my name is John, and I am 48 years old


'Hello, my name is John, and I am 48 years old'

### This works, but it is a bit tedious if we have a lot of methods in the Person class since we then need to decorate each of them with the func_logger decorator. 
### Instead we can create a class decorator

In [10]:
from functools import wraps

def func_logger(fn):
    
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner


def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

@class_logger            
class Person:
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def greet(self):
        return f'Hello, my name is {self.name}, and I am {self.age} years old'

print('\nInstantiating the class...') 
p = Person('John', 48)
p.greet()

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet

Instantiating the class...
Log: Person.__init__((<__main__.Person object at 0x000001D083ECD3A0>, 'John', 48), {}) = None
Log: Person.greet((<__main__.Person object at 0x000001D083ECD3A0>,), {}) = Hello, my name is John, and I am 48 years old


'Hello, my name is John, and I am 48 years old'

### The approach above works for instancemethods, but won't work for classmethods and staticmethods

In [15]:
from functools import wraps

def func_logger(fn):
    
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

def class_logger(cls):
    for name, obj in vars(cls).items():
        
        if callable(obj):
            print('decorating callable', cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        
        elif isinstance(obj, staticmethod):
            print('decorating static method: ', cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        
        elif isinstance(obj, classmethod):
            print('decorating class method: ', cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
        
        elif isinstance(obj, property):
            print('decorating property: ', cls, name)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
                
            setattr(cls, name, obj)
            
    return cls


@class_logger
class Person:
    
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    @staticmethod
    def static_method():
        print('static_method called...')
    
    @classmethod
    def class_method(cls):
        print('class_method called...')
        
    def instance_method(self):
        print('instance_method called...')


p = Person('John')
print()
p.static_method()
p.class_method()
p.instance_method()
p.name

decorating callable <class '__main__.Person'> __init__
decorating property:  <class '__main__.Person'> name
decorating static method:  <class '__main__.Person'> static_method
decorating class method:  <class '__main__.Person'> class_method
decorating callable <class '__main__.Person'> instance_method
Log: Person.__init__((<__main__.Person object at 0x00000126867AF4C0>, 'John'), {}) = None

static_method called...
Log: Person.static_method((), {}) = None
class_method called...
Log: Person.class_method((<class '__main__.Person'>,), {}) = None
instance_method called...
Log: Person.instance_method((<__main__.Person object at 0x00000126867AF4C0>,), {}) = None
Log: Person.name((<__main__.Person object at 0x00000126867AF4C0>,), {}) = John


'John'

### This works well for the methods, but won't work for properties

In [17]:
import inspect

class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    class Other:
        def __call__(self):
            pass
        
    other = Other()
    

keys = ('static_method', 'cls_method', 'inst_method', 'name', '__add__', 'Other', 'other')
inspect_funcs = ('isroutine', 'ismethod', 'isfunction', 'isbuiltin', 'ismethoddescriptor')
    
max_header_length = max(len(key) for key in keys)
max_fname_length = max(len(func) for func in inspect_funcs)
print(format('', f'{max_fname_length}s'), '\t'.join(format(key, f'{max_header_length}s') for key in keys))
for inspect_func in inspect_funcs:
    fn = getattr(inspect, inspect_func)
    inspect_results = (format(str(fn(MyClass.__dict__[key])), f'{max_header_length}s') for key in keys)
    print(format(inspect_func, f'{max_fname_length}s'), '\t'.join(inspect_results))

                   static_method	cls_method   	inst_method  	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	False        	False        
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         False        	False        	True         	False        	True         	False        	False        
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor True         	True         	False        	False        	False        	False        	False        


## Cleaning up the class_logger

In [25]:
from functools import wraps
import inspect

def func_logger(fn):
    
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'Log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

def class_logger(cls):
    for name, obj in vars(cls).items():        
        if isinstance(obj, (staticmethod, classmethod)):
            print(f'Decorating a {type(obj).__name__}')
            type_ = type(obj)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
        
        elif isinstance(obj, property):
            print('decorating property: ', cls, name)
            methods = (('fget', 'getter'), ('fset', 'setter'), ('fdel', 'deleter'))
            for prop, method in methods:
                if getattr(obj, prop):
                    obj = getattr(obj, method)(func_logger(getattr(obj, prop)))
            setattr(cls, name, obj)
            
        elif inspect.isroutine(obj):
            print('decorating callable', cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
            
    return cls


@class_logger
class Person:
    
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    @staticmethod
    def static_method():
        print('static_method called...')
    
    @classmethod
    def class_method(cls):
        print('class_method called...')
        
    def instance_method(self):
        print('instance_method called...')


p = Person('John')
print()
p.static_method()
p.class_method()
p.instance_method()
p.name


decorating callable <class '__main__.Person'> __init__
decorating property:  <class '__main__.Person'> name
Decorating a staticmethod
Decorating a classmethod
decorating callable <class '__main__.Person'> instance_method
Log: Person.__init__((<__main__.Person object at 0x00000126867EC7C0>, 'John'), {}) = None

static_method called...
Log: Person.static_method((), {}) = None
class_method called...
Log: Person.class_method((<class '__main__.Person'>,), {}) = None
instance_method called...
Log: Person.instance_method((<__main__.Person object at 0x00000126867EC7C0>,), {}) = None
Log: Person.name((<__main__.Person object at 0x00000126867EC7C0>,), {}) = John


'John'

# Decorator classes

In [28]:
from types import MethodType

class Logger:
    
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print(f'Log: {self.fn.__name__} called')
        return self.fn(*args, **kwargs)
    
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return MethodType(self, instance)
        
class Person:
    
    def __init__(self, name):
        self.name = name
    
    @Logger
    def say_hello(self):
        return f'{self.name} says hello'

@Logger
def say_bye(name):
    return f'{name }says bye'

   
p = Person('Alex')
print(p.say_hello, end='\n\n')
p.say_hello() 
print()
say_bye('John')

__get__ called: self=<__main__.Logger object at 0x00000126867C5190>, instance=<__main__.Person object at 0x00000126867C57C0>
	returning self as a method to instance
<bound method ? of <__main__.Person object at 0x00000126867C57C0>>

__get__ called: self=<__main__.Logger object at 0x00000126867C5190>, instance=<__main__.Person object at 0x00000126867C57C0>
	returning self as a method to instance
Log: say_hello called

Log: say_bye called


'Johnsays bye'

This is a number


## Metaclasses vs Class Decorators
- As we have seen, class decorators can achieve a lot of the metaprogramming goals we might have.
- But there is one area where they fall short of metaclasses - inheritance.
- Metaclasses are carried through inheritance, whereas decorators are not.

In [29]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner    

def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'
    
Person('Alex', 10).greet()

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet
log: Person.__init__((<__main__.Person object at 0x00000126867A19D0>, 'Alex', 10), {}) = None
log: Person.greet((<__main__.Person object at 0x00000126867A19D0>,), {}) = Hello, my name is Alex and I am 10


'Hello, my name is Alex and I am 10'

## We could do this with a metaclass too:

In [30]:
class ClassLogger(type):
    def __new__(mcls, name, bases, class_dict):
        new_cls = super().__new__(mcls, name, bases, class_dict)
        for key, obj in vars(new_cls).items():
            if callable(obj):
                setattr(new_cls, key, func_logger(obj))
        return new_cls   
    
class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}' 
    
p = Person('John', 78).greet()

log: Person.__init__((<__main__.Person object at 0x00000126867F3970>, 'John', 78), {}) = None
log: Person.greet((<__main__.Person object at 0x00000126867F3970>,), {}) = Hello, my name is John and I am 78


## The main advantage of using a metaclass over a decorator is that it allows for **`inheritance`**

### When we use a decorator we to remember to also decorate the sub-class like below

In [31]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner    

def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'


@class_logger
class Student(Person):
    def __init__(self, name, age, student_number):
        super().__init__(name, age)
        self.student_number = student_number
        
    def study(self):
        return f'{self.name} studies...'
    
s = Student('Alex', 19, 'abcdefg')
print(s.study())
print(s.greet())

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet
decorating: <class '__main__.Student'> __init__
decorating: <class '__main__.Student'> study
log: Person.__init__((<__main__.Student object at 0x00000126867F3160>, 'Alex', 19), {}) = None
log: Student.__init__((<__main__.Student object at 0x00000126867F3160>, 'Alex', 19, 'abcdefg'), {}) = None
log: Student.study((<__main__.Student object at 0x00000126867F3160>,), {}) = Alex studies...
Alex studies...
log: Person.greet((<__main__.Student object at 0x00000126867F3160>,), {}) = Hello, my name is Alex and I am 19
Hello, my name is Alex and I am 19


In [ ]:
class ClassLogger(type):
    def __new__(mcls, name, bases, class_dict):
        new_cls = super().__new__(mcls, name, bases, class_dict)
        for key, obj in vars(new_cls).items():
            if callable(obj):
                setattr(new_cls, key, func_logger(obj))
        return new_cls   

class Person(metaclass=ClassLogger):
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'
    
class Student(Person):
    def __init__(self, name, age, student_number):
        super().__init__(name, age)
        self.student_number = student_number
        
    def study(self):
        return f'{self.name} studies...'
    
s = Student('Alex', 19, 'abcdefg')
s.study()